In [1]:
import findspark
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType
from pyspark.sql.functions import col
from pyspark.sql.functions import desc
from pyspark.storagelevel import StorageLevel
from pyspark.sql.types import IntegerType

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

23/07/27 09:17:24 WARN Utils: Your hostname, RNTDELL000700 resolves to a loopback address: 127.0.1.1; using 192.168.1.49 instead (on interface wlp0s20f3)
23/07/27 09:17:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/27 09:17:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/27 09:17:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
sc = spark.sparkContext

# Spark SQL

* En Spark 1.6 se introdujo una nueva abstracción de programación llamada API estructurada. Esta es la forma preferida para realizar el procesamiento de datos en la mayoría de los casos de uso.

* En esta nueva forma de hacer el procesamiento de datos, los datos deben organizarse en un formato estructurado y la lógica de cálculo de datos debe seguir una determinada estructura. Con estas dos piezas de información, Spark puede realizar optimizaciones para acelerar las aplicaciones de procesamiento de datos.

* El componente SparkSQL está construido sobre el viejo y confiable componente SparkCore. Esta arquitectura en capas significa que cualquier mejora en el componente Spark Core estará disponible automáticamente para el componente para SQL.

* El concepto DF se inspiró en el concepto de pandasDF de python. La principal diferencia es que un DF en Spark puede manejar un gran volumen de datos que se distribuyen en muchas máquinas.

* Un concepto fundamental que diferencia a los datos estructurados de los no estructurados es un **esquema** que define la estructura de los datos en forma de nombres de columna y tipos de datos asociados. El concepto de **esquema** es una parte integral de las APIs estructuradas de Spark.

* Los datos estructurados a menudo se capturan en un formato determinado. Algunos de estos formatos están basados en textos y algunos de ellos están basados en binario.

    * Los formatos comunes para datos de texto son CSV, XML y JSON 
    * Los formatos comunes para datos binarios son agro, parquet y ORC.
<br>.
* El módulo SparkSQL y facilita la lectura y escritura de datos desde y hacia cualquiera de estos formatos. Un beneficio inesperado que surge de esta versatilidad es que Spark se puede utilizar como una herramienta de conversión de formato de datos.

## 1. Creación de DFs

### 1-1. Crear un DF a partir de un RDD

Hay muchas formas de crear un DF, pero **siempre hay que proporcionar un esquema**, ya sea implícita o explícitamente.

In [3]:
rdd = sc.parallelize([item for item in range(10)]).map(lambda x: (x, x ** 2))

rdd.collect()

[(0, 0),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81)]

In [4]:
# Para crear un DF a parti de un RDD no tenemos más que pasarle a la función .toDF los nombres de las columnas como lista
df = rdd.toDF(['numero', 'cuadrado'])

# Podemos ver el esquema de los datos, que nos informa del tipo de datos y de si acepta nulos o no
df.printSchema()

# Podemos ver el DF
df.show()

root
 |-- numero: long (nullable = true)
 |-- cuadrado: long (nullable = true)

+------+--------+
|numero|cuadrado|
+------+--------+
|     0|       0|
|     1|       1|
|     2|       4|
|     3|       9|
|     4|      16|
|     5|      25|
|     6|      36|
|     7|      49|
|     8|      64|
|     9|      81|
+------+--------+



##### También podemos crear un DF a partir de un RDD definiendo nosotros el esquema

In [5]:
rdd1 = sc.parallelize([(1, 'Jose', 35.5), (2, 'Teresa', 54.3), (3, 'Katia', 12.7)])

In [6]:
# Método 1: creamos el esquema a partir de las classes de pyspark.sql.types

esquema1 = StructType(
    [
     StructField('id', IntegerType(), True),
     StructField('nombre', StringType(), True),
     StructField('saldo', DoubleType(), True)
    ]
)

In [7]:
# Método 2: creamos el esquema a partir de un string

esquema2 = "`id` INT, `nombre` STRING, `saldo` DOUBLE"

In [8]:
# Finalmente, para crear un DF aplicando nuestro esquema, usamos la función .createDtaFrame, pasándole el RDD y el esquema

df1 = spark.createDataFrame(rdd1, schema=esquema1)

df1.printSchema()

df1.show()

root
 |-- id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- saldo: double (nullable = true)

+---+------+-----+
| id|nombre|saldo|
+---+------+-----+
|  1|  Jose| 35.5|
|  2|Teresa| 54.3|
|  3| Katia| 12.7|
+---+------+-----+



In [9]:
df1 = spark.createDataFrame(rdd1, schema=esquema2)

df1.printSchema()

df1.show()

root
 |-- id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- saldo: double (nullable = true)

+---+------+-----+
| id|nombre|saldo|
+---+------+-----+
|  1|  Jose| 35.5|
|  2|Teresa| 54.3|
|  3| Katia| 12.7|
+---+------+-----+



### 1-2. Crear un DF a partir de fuentes de datos

Las dos clases principales en SparkSQL para leer y escribir datos son DataFrameReader y DataFrameWriter respectivamente.

#### DataFrameReader

* Una instancia de la clase DataFrameReader está disponible como read en la sesión de Spark y la podemos invocar a través de **spark.read**

* El patrón común para interactuar con DataFrameReader es **spark.read.format(...).option('key', 'value').schema(...).load()**

    * **.format(...)** no es opcional. Puede ser una de las fuente de datos integradas o un formato de dato personalizado

        * **Formato integrado:** se puede usar un nombre corto como por ejemplo json, parquet, jdbc, orc, csv, text...
        * **Formato personalizado:** debe proporcionar un nombre completo
<br>.

    * **.option('key', 'value')** es opcional porque DataFrameReader tiene un conjunto de opciones predeterminadas para cada formato de fuente de datos. Podemos anular estos valores predeterminados proporcionando un valor a la función option('key', 'value')

    * **.schema(...)** puede ser opcional porque algunas fuentes de datos tienen el esquema incrustado dentro de los archivos de datos, podemos pensar en .parquet u .orc En estos casos el esquema se infiere automáticamente, para otros casos es posible que deba proporcionar un esquema.

* Para leer los datos hay dos alternativas aplicables a todos los formatos:

    * **spark.read.<extension>("<path>")** por ejemplo, spark.read.json("/path/to/file.json")
    * **spark.read.format("<extension>").load("<path>")** por ejemplo spark.read.format("json").load("/path/to/file.json")



In [10]:
# Crear un DF a partir de un archivo de texto
dftxt = spark.read.text('./data/data_DF/dataTXT.txt')

dftxt.show(truncate=False)

+-----------------------------------------------------------------------+
|value                                                                  |
+-----------------------------------------------------------------------+
|Estamos en el curso de pyspark                                         |
|En este capítulo estamos estudiando el API SQL de Saprk                |
|En esta sección estamos creado dataframes a partir de fuentes de datos,|
|y en este ejemplo creamos un dataframe a partir de un texto plano      |
+-----------------------------------------------------------------------+



In [11]:
# Crear un DataFrame mediante la lectura de un archivo csv

dfcsv = spark.read.csv('./data/data_DF/dataCSV.csv')

# Aquí mete el nombre de las columnas en la primera fila y da nombres nuevos predeterminados
dfcsv.show(n=5)

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+--------------------+--------------------+
|        _c0|          _c1|                 _c2|                 _c3|        _c4|                 _c5|                 _c6|    _c7|   _c8|     _c9|         _c10|                _c11|             _c12|            _c13|                _c14|                _c15|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+--------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|vid

In [12]:
# Para que tome la primera fila como nombre de columnas:
dfcsv1 = spark.read.option('header', 'true').csv('./data/data_DF/dataCSV.csv')

dfcsv1.show(n=5)

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [13]:
# Leer un archivo de texto con un delimitador diferente

dftxt1 = spark.read.option('header', 'true').option('delimiter', '|').csv('./data/data_DF/dataTab.txt')

dftxt1.show()


+----+----+----------+-----+
|pais|edad|     fecha|color|
+----+----+----------+-----+
|  MX|  23|2021-02-21| rojo|
|  CA|  56|2021-06-10| azul|
|  US|  32|2020-06-02|verde|
+----+----+----------+-----+



In [14]:
# Crear un DataFrame a partir de un json proporcionando un schema
json_schema =  StructType(
    [
     StructField('color', StringType(), True),
     StructField('edad', IntegerType(), True),
     StructField('fecha', DateType(), True),
     StructField('pais', StringType(), True)
    ]
)

dfjson = spark.read.schema(json_schema).json('./data/data_DF/dataJSON.json')

dfjson.show()

dfjson.printSchema()

+-----+----+----------+----+
|color|edad|     fecha|pais|
+-----+----+----------+----+
| rojo|null|2021-02-21|  MX|
| azul|null|2021-06-10|  CA|
|verde|null|2020-06-02|  US|
+-----+----+----------+----+

root
 |-- color: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- fecha: date (nullable = true)
 |-- pais: string (nullable = true)



In [15]:
# Crear un DataFrame a partir de un archivo parquet
dfparquet = spark.read.parquet('./data/data_DF/dataPARQUET.parquet')

dfparquet.show(n=5)

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [16]:
# Otra alternativa para leer desde una fuente de datos parquet en este caso
dfparquet1 = spark.read.format('parquet').load('./data/data_DF/dataPARQUET.parquet')

dfparquet1.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



## 2. Trabajo con columnas

* A diferencia de las operaciones con RDD, las operaciones estructuradas están diseñadas para ser más relacionales, lo que significa que estas operaciones reflejan el tipo de expresiones que pueden hacer con SQL como filtrado, transformación, unión, entre otras.

* Al igual que las operaciones con RDD, las operaciones estructuradas se dividen en dos categorías: transformaciones y acciones. La semántica de las transformaciones y acciones estructuradas es idéntica a la de los RDD: las transformaciones estructuradas son **lazy evaluation** y las acciones estructuradas son **eager evaluation**.

* Recordmeos que los DF son inmutables y sus operaciones de transformación siempre devuelven un nuevo DF.

In [17]:
dfparquet2 = spark.read.parquet('./data/dataPARQUET.parquet')

dfparquet2.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [18]:
# Primera alternativa para referirnos a las columnas

dfparquet2.select('title').show(5)

+--------------------+
|               title|
+--------------------+
|WE WANT TO TALK A...|
|The Trump Preside...|
|Racist Superman |...|
|Nickelback Lyrics...|
|I Dare You: GOING...|
+--------------------+
only showing top 5 rows



In [19]:
# Segunda alternativapara referirnos a las columnas

dfparquet2.select(col('title')).show(5)

+--------------------+
|               title|
+--------------------+
|WE WANT TO TALK A...|
|The Trump Preside...|
|Racist Superman |...|
|Nickelback Lyrics...|
|I Dare You: GOING...|
+--------------------+
only showing top 5 rows



## 3. Transformaciones


### 3-1. Funciones select y selectExpr

* **select:** esta transformación se usa comúnmente para realizar la proyección, es decir, seleccionar todas o un subconjunto de columnas de un DF. Durante la selección cada columna se puede transformar mediante una expresión de columna.

In [20]:
# Cargamos el parquet, muy similar al anterior, pero con alguna modificación en el tipo de datos de cada columna
dfparquet3 = spark.read.parquet('./data/datos.parquet')

dfparquet3.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [21]:
dfparquet3.select(col('video_id')).show(5)

+-----------+
|   video_id|
+-----------+
|2kyS6SvSYSE|
|1ZAPwfrtAFY|
|5qpjK5DgCt4|
|puqaWrEC7tY|
|d380meD0W0M|
+-----------+
only showing top 5 rows



In [22]:
dfparquet3.select('video_id', 'trending_date').show(5)
# El problema de esta nomenclatura es que no podemos pasarle funciones, como se puede comprobar en la siguiente celda

+-----------+-------------+
|   video_id|trending_date|
+-----------+-------------+
|2kyS6SvSYSE|     17.14.11|
|1ZAPwfrtAFY|     17.14.11|
|5qpjK5DgCt4|     17.14.11|
|puqaWrEC7tY|     17.14.11|
|d380meD0W0M|     17.14.11|
+-----------+-------------+
only showing top 5 rows



In [23]:
# Esta expresión nos dará error

dfparquet3.select(
    'likes',
    'dislikes',
    ('likes' - 'dislikes')
).show(5)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
# La forma correcta es usando la función col

dfparquet3.select(
    col('likes'),
    col('dislikes'),
    (col('likes') - col('dislikes')).alias('aceptacion')
).show(5)

+------+--------+----------+
| likes|dislikes|aceptacion|
+------+--------+----------+
| 57527|    2966|     54561|
| 97185|    6146|     91039|
|146033|    5339|    140694|
| 10172|     666|      9506|
|132235|    1989|    130246|
+------+--------+----------+
only showing top 5 rows



* **selectExpr:** nos permite aplicar una expresión a la selección de columnas

In [ ]:
# Le pedimos los mismo que en la celda anterior
dfparquet3.selectExpr('likes', 'dislikes', '(likes - dislikes) as aceptacion').show(5)

+------+--------+----------+
| likes|dislikes|aceptacion|
+------+--------+----------+
| 57527|    2966|     54561|
| 97185|    6146|     91039|
|146033|    5339|    140694|
| 10172|     666|      9506|
|132235|    1989|    130246|
+------+--------+----------+
only showing top 5 rows



In [ ]:
# Contamos el número de valores diferentes dentro de la columna video_id
dfparquet3.selectExpr("count(distinct(video_id)) as videos").show()

+------+
|videos|
+------+
|  6837|
+------+



>\
>Como conclusión, deberíamos tener en cuenta que las expresiones SQL son construcciones poderosas y flexibles que permiten expresar la lógica de transformación de columnas de una manera natural, tal como lo podríamos estar pensando nosotros mismos. Podemos expresar expresiones SQL en un formato de cadena y spark las analizará en un árbol lógico que se evalúa en un orden correcto. Además hay que tener en cuenta que la combinación de expresiones SQL y funciones integradas facilita la realización de ciertos análisis de datos que de otro modo requerirían varios pasos.
>
><br>

### 3-2. Funciones filter y where

* **filter**

In [ ]:
dfparquet3.show(5)

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13 18:13:01|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           False| 

In [ ]:
dfparquet3.filter(col('video_id') == '2kyS6SvSYSE').show()

+-----------+-------------+--------------------+-------------+-----------+-------------------+---------------+-------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|channel_title|category_id|       publish_time|           tags|  views|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+-------------+-----------+-------------------+---------------+-------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...| CaseyNeistat|         22|2017-11-13 18:13:01|SHANtell martin| 748374|57527|    2966|        15954|https://i.ytimg.c...|            False|           False|                 False|SHANTELL'S CHANNE...|
|2kyS6Sv

* **where**

In [ ]:
dfparquet4 = spark.read.parquet('./data/datos.parquet').where(col('trending_date') != '17.14.11')

dfparquet4.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|            video_id|       trending_date|               title|       channel_title|         category_id|       publish_time|                tags| views|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|\nCook with confi...|             recipes|              videos| and restaurant g...| dining destinations|               null|                null| 

In [ ]:
dfparquet5 = spark.read.parquet('./data/datos.parquet').where(col('likes') > 5000)

dfparquet5.show(5)

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13 18:13:01|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           False| 

In [ ]:
dfparquet5.filter((col('trending_date') != '17.14.11') & (col('likes') > 7000)).show(5)

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|  views|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|YvfYK0EEhK4|     17.15.11|Brent Pella - Why...|         Brent Pella|         23|2017-11-14 16:32:51|"spirit airlines"...| 462490|14132|     795|          666|https://i.ytimg.c...|            False|           False|     

In [ ]:
# Esto da exactamente el mismo resultado que la celda anterior, pero usando filter
dfparquet5.filter(col('trending_date') != '17.14.11').filter(col('likes') > 7000).show(5)

### 3-3. Funciones distinc y dropDuplicates

Estas dos transformaciones tienen un comportamiento idéntico sin embargo, **dropDuplicates** nos permitirá controlar qué columnas deben usarse en la lógica de deduplicación. Si no especificamos ninguna columna, la lógica de deduplicación utilizará todas las columnas del DF.

* **distinc** elimina todos los duplicados de todas las columnas

In [ ]:
df_sin_duplicados = dfparquet3.distinct()

print('El conteo del dataframe original es {}'.format(dfparquet3.count()))
print('El conteo del dataframe sin duplicados es {}'.format(df_sin_duplicados.count()))

El conteo del dataframe original es 48137
El conteo del dataframe sin duplicados es 41428


* **dropDuplicates** elimina los duplicados en las columnas que le indiquemos en forma de lista. Si no indicamos ninguna, hace lo mismo que distinc

In [ ]:
dataframe = spark.createDataFrame([(1, 'azul', 567), (2, 'rojo', 487), (1, 'azul', 345), (2, 'verde', 783)]).toDF('id', 'color', 'importe')

dataframe .show()

+---+-----+-------+
| id|color|importe|
+---+-----+-------+
|  1| azul|    567|
|  2| rojo|    487|
|  1| azul|    345|
|  2|verde|    783|
+---+-----+-------+



In [ ]:
# Elimina los registros cuyas columnas id y color sean idénticas
dataframe.dropDuplicates(['id', 'color']).show()

+---+-----+-------+
| id|color|importe|
+---+-----+-------+
|  1| azul|    567|
|  2| rojo|    487|
|  2|verde|    783|
+---+-----+-------+



### 3-4. Funciones sort, orderBy y limit

sort y orderby tienen la misma semántica pero la transformación orderBy es más relacional que la de sort.
De forma predeterminada, el orden está dado en forma ascendente y es bastante fácil cambiarlo a descendente.
Al especificar más de una columna, es posible tener un orden diferente para cada una de las columnas.

Con limit elegimos cuántas filas queremos mostrar.

In [ ]:
# Leemos el DF con al de procesamiento incluído en la carga
dfparquet6 = (spark.read.parquet('./data/datos.parquet')
    .select(col('likes'), col('views'), col('video_id'), col('dislikes'))
    .dropDuplicates(['video_id'])
)

dfparquet6.show(5)

+-----+-------+-----------+--------+
|likes|  views|   video_id|dislikes|
+-----+-------+-----------+--------+
|63995|1525400|bAkEd8r7Nnw|     896|
|  427|   9036|eijd-yjXY9E|      14|
| 4145| 318249|npcqBt_e4k0|     110|
| 6669| 203615|LeWtF5y9-6Q|     136|
| 2166| 104499|GhcqN2FDAnA|    1066|
+-----+-------+-----------+--------+
only showing top 5 rows



In [ ]:
# sort va a ordenar el DF por la columna like de forma ascendente

dfparquet6.sort('likes').show(5)

+-----+-----+--------------------+--------+
|likes|views|            video_id|dislikes|
+-----+-----+--------------------+--------+
| null| null|\nMust-See WWE vi...|    null|
| null| null|Horror Outro ► ht...|    null|
| null| null|             \nToday|    null|
| null| null|\nhttp://www.Mast...|    null|
| null| null|\nSIGN UP FOR BRA...|    null|
+-----+-----+--------------------+--------+
only showing top 5 rows



In [ ]:
# sort va a ordenar el DF por la columna like de forma descendente

dfparquet6.sort(desc('likes')).show(5)

+-------+--------+-----------+--------+
|  likes|   views|   video_id|dislikes|
+-------+--------+-----------+--------+
|3880071|39349927|7C2z4GqqS5E|   72707|
|2055137|13945717|kTlv5_Bs8aw|   23888|
|2050527|10695328|OK3GJ0WIQ8s|   14711|
|1956202|10666323|p8npDG2ulKQ|   13966|
|1735895|37736281|6ZfuNTqbHE8|   21969|
+-------+--------+-----------+--------+
only showing top 5 rows



In [ ]:
# orderBy

dfparquet6.orderBy(col('views')).show(5)

+-----+-----+--------------------+--------+
|likes|views|            video_id|dislikes|
+-----+-----+--------------------+--------+
| null| null|             \nToday|    null|
| null| null|\nBIG SHINY Lens ...|    null|
| null| null|\nMust-See WWE vi...|    null|
| null| null|\nSIGN UP FOR BRA...|    null|
| null| null|          \nEvelin 7|    null|
+-----+-----+--------------------+--------+
only showing top 5 rows



In [ ]:
dfparquet6.orderBy(col('views').desc()).show(5)

+-------+--------+-----------+--------+
|  likes|   views|   video_id|dislikes|
+-------+--------+-----------+--------+
| 609101|48431654|-BQJo3vK8O8|   52259|
|3880071|39349927|7C2z4GqqS5E|   72707|
|1111592|38873543|i0p1bmr0EmE|   96407|
|1735895|37736281|6ZfuNTqbHE8|   21969|
|1634124|33523622|2Vv-BfVoq4g|   21082|
+-------+--------+-----------+--------+
only showing top 5 rows



In [ ]:
dataframe2 = spark.createDataFrame([(1, 'azul', 568), (2, 'rojo', 235), (1, 'azul', 456), (2, 'azul', 783)]).toDF('id', 'color', 'importe')

dataframe2.show()

+---+-----+-------+
| id|color|importe|
+---+-----+-------+
|  1| azul|    568|
|  2| rojo|    235|
|  1| azul|    456|
|  2| azul|    783|
+---+-----+-------+



In [ ]:
# Ahora ordenaremos la columna color en orden descendente e importe ascendente
dataframe2.orderBy(col('color').desc(), col('importe')).show()

+---+-----+-------+
| id|color|importe|
+---+-----+-------+
|  2| rojo|    235|
|  1| azul|    456|
|  1| azul|    568|
|  2| azul|    783|
+---+-----+-------+



In [ ]:
# limit para visualizar el top 10 de videos con más visualizaciones

top_10 = dfparquet6.orderBy(col('views').desc()).limit(10)

top_10.show()

+-------+--------+-----------+--------+
|  likes|   views|   video_id|dislikes|
+-------+--------+-----------+--------+
| 609101|48431654|-BQJo3vK8O8|   52259|
|3880071|39349927|7C2z4GqqS5E|   72707|
|1111592|38873543|i0p1bmr0EmE|   96407|
|1735895|37736281|6ZfuNTqbHE8|   21969|
|1634124|33523622|2Vv-BfVoq4g|   21082|
|1405355|31648454|VYOjWnS4cMY|   51547|
| 850362|27973210|u9Mv98Gr5pY|   26541|
|1149185|24782158|FlsCjmMhFmw|  483924|
| 641546|24421448|U9BwWKXjVaI|   16517|
| 587326|23758250|1J76wN0TPI4|   18799|
+-------+--------+-----------+--------+



In [ ]:
# Vamos a ver los videos más odiados de youtube
topmierder = dfparquet6.orderBy(col('dislikes').desc()).limit(10)

topmierder.show()

+-------+--------+-----------+--------+
|  likes|   views|   video_id|dislikes|
+-------+--------+-----------+--------+
| 835378|13305605|QwZT7T-TXT0|  629120|
|1149185|24782158|FlsCjmMhFmw|  483924|
|1207457|13754992|_5d-sQ7Fh5M|  280675|
|   9100| 1142585|LFhT6H6pRWg|  218841|
| 285349| 6886948|ooyjaVdt-jA|  164004|
|1167488| 8041970|oWjxSkJpxFU|  147643|
|  32892|14647590|V5cOvyDpWfM|  117128|
|   4870|  985179|8d_202l55LU|  110707|
|1111592|38873543|i0p1bmr0EmE|   96407|
| 761852| 7691902|khPLWaBioOs|   86475|
+-------+--------+-----------+--------+



### 3-5. Funciones whithColumn y withColumnRenamed

* **whithColumn** se usa para agregar una nueva columna a un DF. Requiere dos parámetros de entrada, un nombre de columna y un valor en forma de expresión de columna.

In [ ]:
# Vamos a crear una nueva columna con whithColumn
df_valoracion = dfparquet3.withColumn('valoracion', col('likes') - col('dislikes'))

df_valoracion.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)
 |-- valoracion: integer (nullable = true)



In [ ]:
df_valoracion1 = (dfparquet3.withColumn('valoracion', col('likes') - col('dislikes'))
                    .withColumn('res_div', col('valoracion') % 10)
)

df_valoracion1.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)
 |-- valoracion: integer (nullable = true)
 |-- res_div: integer (nullable = true)



In [ ]:
df_valoracion1.select(col('likes'), col('dislikes'), col('valoracion'), col('res_div')).show(5)

+------+--------+----------+-------+
| likes|dislikes|valoracion|res_div|
+------+--------+----------+-------+
| 57527|    2966|     54561|      1|
| 97185|    6146|     91039|      9|
|146033|    5339|    140694|      4|
| 10172|     666|      9506|      6|
|132235|    1989|    130246|      6|
+------+--------+----------+-------+
only showing top 5 rows



* **withColumnRenamed** se usa para cambiar de nombre una columna. Recibe dos parámetros, el nombre actual y el nuevo nombre. Si el nombre de la columna actual proporciobnado no existe, sprak no arroja error, simplemente no hace nada

In [ ]:
df_renombrado = dfparquet3.withColumnRenamed('video_id', 'id')

df_renombrado.printSchema()

root
 |-- id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [ ]:
# Le pasamos un nombre incorrecto y no da error, pero no hace nada
df_error = dfparquet3.withColumnRenamed('nombre_que_no_existe', 'otro_nombre')

df_error.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



### 3-6. Funciones drop, sample y randomSplit

* **drop** elimina las columnas especificadas del data frame. Pueden especificarse uno o más nombres de columna para eliminar, pero solo se eliminarán las que existan en el esquema y las que no se ignorarán.

In [ ]:
df_util = dfparquet3.drop('comments_disabled')

df_util.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [ ]:
df_util1 = dfparquet3.drop('comments_disabled', 'ratings_disabled', 'thumbnail_link')

df_util1.printSchema()


root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [ ]:
# Aunque le pasemos una  columna que no existe, en este caso 'cafe', no arrojará error, simplemente la ignorará
df_util = dfparquet3.drop('comments_disabled', 'ratings_disabled', 'thumbnail_link', 'cafe')

df_util.printSchema()


root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



* **sample** es una transformación que devuelve un conjunto de filas seleccionados aleatoriamente del DF. El número de filas devuelto será aproximadamente igual al número especificado, que representa un porcentaje y es un valor que debe oscilar entre cero y uno.

In [ ]:
df_muestra = dfparquet3.sample(0.8)

In [ ]:
num_filas = dfparquet3.count()
num_filas_muestra = df_muestra.count()

print('El 80% de filas del dataframe original es {}'.format(num_filas - (num_filas*0.2)))
print('El numero de filas del dataframe muestra es {}'.format(num_filas_muestra))

El 80% de filas del dataframe original es 38509.6
El numero de filas del dataframe muestra es 38628


>\
>Aunque **sample** es un muestreo aleatorio, podemos hacer que repita siempre la misma selección si le pasamos un seed
>
><br>

In [ ]:
df_muestra1 = dfparquet3.sample(fraction=0.8, seed=1234)

>\
>A **sample** también podemos indicarle si queremos que tome las filas del DF con reemplazo o no
>
><br>

In [ ]:
df_muestra = dfparquet3.sample(withReplacement=True, fraction=0.8, seed=1234)

* **randomSplit** es una transformación que se usa comúnmente durante el proceso de preparación de los datos para entrenar modelos de ML.

A diferencia de las transformaciones anteriores, esta nos va a devolver uno o más data frame y la cantidad de data free que devuelve se depende de la cantidad de pesos que especifiquemos. Si el conjunto de pesos proporcionados no suma uno, estos pesos se normalizan en consecuencia para sumar uno.

In [ ]:
# Vamos a asignar el 80% de las filas al DF train y el 20% al DF test
train, test = dfparquet3.randomSplit([0.8, 0.2], seed=1234)

In [ ]:
print(train.count())
print(test.count())

38542
9595


In [ ]:
# Vamos a asignar el 60% de las filas al DF train1, el 20% al validation1 y el 20% al DF test1
train1, validation1, test1 = dfparquet3.randomSplit([0.6, 0.2, 0.2], seed=1234)

In [ ]:
print(train1.count())
print(validation1.count())
print(test1.count())

28799
9743
9595


## 4. Trabajo con datos incorrectos o faltantes

* Los datos con los que trabajamos a menudo no están tan limpio como nos gustaría. En parte esto se debe a que los datos evolucionan con el tiempo y por lo tanto algunas columnas tienen valores y otras no. Es importante abordar este tipo de problema al comienzo de la lógica de la manipulación de datos para evitar sorpresas desagradables que harán que su trabajo de procesamiento de datos de larga duración deje de funcionar.

* La comunidad de spark reconoce que la necesidad de lidiar con datos faltantes es una realidad. Por lo tanto, proporciona una clase dedicada para ayudar a afrontar este problema.

### Formas más habituales de tratar los datos faltantes

* Eliminar las filas que tienen valores perdidos en una o más columnas.

* Llenar esos valores faltantes con valores proporcionados por el usuario.

### 4-1. Eliminar nulos

* Podemos eliminar **todos** los valores nulos del DF

In [ ]:
# Primero vamos a ver el número de registros del DF
dfparquet3.count()

48137

In [ ]:
# Ahora vamos a eliminar todos los registros que tengan al menos un valor nulo y ver cuánto registros quedan
dfparquet3.na.drop().count()

40379

In [ ]:
# Podemos hacer lo mismo que en la celda anterior de este modo:
dfparquet3.na.drop('any').count()

40379

In [ ]:
# Y aún hay otro método para eliminar nulos:
dfparquet3.dropna().count()

40379

* Podemos eliminar **algunos** de los valores nulos del DF

In [ ]:
# Vamos a eliminar filas que tengan nulos sólo en la columna 'views'
dfparquet3.na.drop(subset=['views']).count()

40949

In [ ]:
# Vamos a eliminar filas que tengan nulos sólo en las columna 'views' y 'dislikes'
dfparquet3.na.drop(subset=['views', 'dislikes']).count()

40949

### 4-2. Rellenar los valores nulos

In [ ]:
# Vemos que hay tres columnas llenas de nulos
dfparquet3.orderBy(col('views')).select(col('views'), col('likes'), col('dislikes')).show(5)

+-----+-----+--------+
|views|likes|dislikes|
+-----+-----+--------+
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
+-----+-----+--------+
only showing top 5 rows



* Podemos rellenar **todos** los nulos

In [ ]:
# Sustituímos los todos los nulos de las columnas seleccionadas por 0 con fillna
dfparquet3.fillna(0).orderBy(col('views')).select(col('views'), col('likes'), col('dislikes')).show(5)

+-----+-----+--------+
|views|likes|dislikes|
+-----+-----+--------+
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
+-----+-----+--------+
only showing top 5 rows



* Podemos rellenar **algunos** nulos

In [ ]:
# Sustituímos por 0 sólo los nulos de las columnas 'likes' y 'dislikes'
dfparquet3.fillna(0, subset=['likes', 'dislikes']).orderBy(col('views')).select(col('views'), col('likes'), col('dislikes')).show(5)

+-----+-----+--------+
|views|likes|dislikes|
+-----+-----+--------+
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
+-----+-----+--------+
only showing top 5 rows



## 5. Acciones sobre un DF en SparkSQL

*  Estas acciones tienen el mismo comportamiento que las acciones realizadas en un RDD, es decir, son **eager evaluation**, por lo que desencadenarán el cálculo de todas las transformaciones acumuladas en el DAG.

### 5-1. show()
Por defecto nos da las primeras 20 filas de un DF y trunca el contenido si supera el ancho máximo de columna por defecto, pero podemos indicarle tanto el número de filas que queremos ver, como que las queremos completas

In [ ]:
dfparquet3.show(5, truncate=False)

+-----------+-------------+--------------------------------------------------------------+---------------------+-----------+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### 5-2. take()
Va a tomas el número de filas que le digamos y nos las devuelve en forma de lista

In [ ]:
dfparquet3.take(1)

[Row(video_id='2kyS6SvSYSE', trending_date='17.14.11', title='WE WANT TO TALK ABOUT OUR MARRIAGE', channel_title='CaseyNeistat', category_id='22', publish_time=datetime.datetime(2017, 11, 13, 18, 13, 1), tags='SHANtell martin', views=748374, likes=57527, dislikes=2966, comment_count=15954, thumbnail_link='https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg', comments_disabled='False', ratings_disabled='False', video_error_or_removed='False', description="SHANTELL'S CHANNEL - https://www.youtube.com/shantellmartin\\nCANDICE - https://www.lovebilly.com\\n\\nfilmed this video in 4k on this -- http://amzn.to/2sTDnRZ\\nwith this lens -- http://amzn.to/2rUJOmD\\nbig drone - http://tinyurl.com/h4ft3oy\\nOTHER GEAR ---  http://amzn.to/2o3GLX5\\nSony CAMERA http://amzn.to/2nOBmnv\\nOLD CAMERA; http://amzn.to/2o2cQBT\\nMAIN LENS; http://amzn.to/2od5gBJ\\nBIG SONY CAMERA; http://amzn.to/2nrdJRO\\nBIG Canon CAMERA; http://tinyurl.com/jn4q4vz\\nBENDY TRIPOD THING; http://tinyurl.com/gw3ylz2\\nYOU NEED T

### 5-3. head()
Toma el número de registros que le indiquemos del comienzo del DF en forma de lista

In [ ]:
dfparquet3.head(1)

[Row(video_id='2kyS6SvSYSE', trending_date='17.14.11', title='WE WANT TO TALK ABOUT OUR MARRIAGE', channel_title='CaseyNeistat', category_id='22', publish_time=datetime.datetime(2017, 11, 13, 18, 13, 1), tags='SHANtell martin', views=748374, likes=57527, dislikes=2966, comment_count=15954, thumbnail_link='https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg', comments_disabled='False', ratings_disabled='False', video_error_or_removed='False', description="SHANTELL'S CHANNEL - https://www.youtube.com/shantellmartin\\nCANDICE - https://www.lovebilly.com\\n\\nfilmed this video in 4k on this -- http://amzn.to/2sTDnRZ\\nwith this lens -- http://amzn.to/2rUJOmD\\nbig drone - http://tinyurl.com/h4ft3oy\\nOTHER GEAR ---  http://amzn.to/2o3GLX5\\nSony CAMERA http://amzn.to/2nOBmnv\\nOLD CAMERA; http://amzn.to/2o2cQBT\\nMAIN LENS; http://amzn.to/2od5gBJ\\nBIG SONY CAMERA; http://amzn.to/2nrdJRO\\nBIG Canon CAMERA; http://tinyurl.com/jn4q4vz\\nBENDY TRIPOD THING; http://tinyurl.com/gw3ylz2\\nYOU NEED T

### 5-3. collect()
Nos trae todos los elementos que le indiquemos en forma de lista, pero hay que usarla con precaución porque si la aplicamos a una cantidad de datos demasiado grande, puede causar un error de desborde de memoria.

In [ ]:
dfparquet3.select('likes').collect()

## 6. Escritura de DFs

En sparkSQK la clase DataFrameWriter es la responsable de la lógica y la complejidad de escribir los datos de un DF en un sistema de almacenamiento externo.\
Una instancia de la clase DataFrameWriter está disponible como variable de escritura en la clase DataFrame.\
El patrón para interactuar con DataFrameWriter es similar al de DataFrameReader:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;df.write.format(...).mode(...).option(...).partitionBy(...).bucketBy(...).sortBy(...).path(<path/to/file>)

El formato predeterminado es .parquet

Las funciones .partitionBy(...).bucketBy(...).sortBy(...) se utilizan para controlar la estructura de los directorios de salida en las fuentes de datos basadas en archivo.

El modo de guardado controla cómo se manejará si la ruta de guardado ya existe y tiene contenido:
 * **append** agrega los datos del DF a la lista de archivos que ya existen en la ubicación de destino especificada
 * **overwrite** sobrescribe completamente cualquier archivo de datos que ya exista en la ubicación de destino especificada con los datos del DF.
 * **error, error if exist, default** es el modo por defecto. Si existe la ubicación de destino especificada DataFrameWriter arrojará un error.
 * **ignore** si existe la ubicación de destino especificada, simplemente no se hará nada.

In [24]:
# Vamos a crear un DF a partir de dfparquet3, con un repartition

df2 = dfparquet3.repartition(2)

In [28]:
# Lo guardamos cambiando el separador por defecto ',' por '|' y creará dos archivos porque el DF tiene dos particiones
df2.write.format('csv').option('sep', '|').save('./output/csv')

In [29]:
# Si queremos que lo guarde en un solo csv, tenemos que dejar el DF en una sola partición
df2.coalesce(1).write.format('csv').option('sep', '|').save('./output/csv1')

In [31]:
dfparquet3.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [33]:
dfparquet3.select('comments_disabled').distinct().show()

+-----------------+
|comments_disabled|
+-----------------+
|            False|
|             null|
| sports and more.|
|          Wiz Kid|
|             True|
|         farfalle|
+-----------------+



In [34]:
df_limpio = dfparquet3.filter(col('comments_disabled').isin('True', 'False'))

In [ ]:
# Vamos a particionar los datos por la columna 'comments_disabled' y guardarlo al mismo tiempo
df_limpio.write.partitionBy('comments_disabled').parquet('./output/parquet')

## 7. Persistencia de DFs

Los DFs se pueden persistir o cachear en la memoria, tal como se hace con los RDD. Los mismos tipos de persistencia están disponibles en la clase de Data Frame.\
Sin embargo, hay una gran diferencia al almacenar en caché un data frame sparkSQL. Sabe el esquema de los datos dentro de un DF, por lo que organiza los datos en un formato de columnas y aplica las compresiones aplicables para minimizar el uso de espacio. Como resultado, se requerirá mucho menos espacio para almacenar un DF en la memoria que para almacenar una RDD cuando ambos están respaldados por el mismo archivo de datos.

In [36]:
df3 = spark.createDataFrame([(1, 'a'), (2, 'b'), (3, 'c')], ['id', 'valor'])

In [38]:
df3.show()

+---+-----+
| id|valor|
+---+-----+
|  1|    a|
|  2|    b|
|  3|    c|
+---+-----+



In [39]:
# Persistimos el DF en memoria
df3.persist()

DataFrame[id: bigint, valor: string]

In [40]:
df3.unpersist()

DataFrame[id: bigint, valor: string]

In [41]:
df3.cache()

DataFrame[id: bigint, valor: string]

Al igual que con los RDD, podemos manejar el nivel del storage, el nivel donde queremos persistir nuestro DF.

In [44]:
df3.persist(StorageLevel.DISK_ONLY)

DataFrame[id: bigint, valor: string]

In [45]:
df3.persist(StorageLevel.MEMORY_AND_DISK)

23/07/24 11:50:53 WARN CacheManager: Asked to cache already cached data.


DataFrame[id: bigint, valor: string]

In [46]:
df3.unpersist()

DataFrame[id: bigint, valor: string]